## Pretprocessing

#### Ucitavamo dataset i pretprocesiramo ga.

In [1]:
import pandas as pd
import spacy
import numpy as np

In [29]:
train_set = pd.read_csv(filepath_or_buffer='./Dataset/train.csv', sep=',')
test_set = pd.read_csv(filepath_or_buffer='./Dataset/test.csv', sep=',')
sample_submission_set = pd.read_csv(filepath_or_buffer='./Dataset/sampleSubmission.csv', sep=',')

In [30]:
print(train_set.head(4))

   id                      query  \
0   1  bridal shower decorations   
1   2       led christmas lights   
2   4                  projector   
3   5                  wine rack   

                                       product_title  \
0        Accent Pillow with Heart Design - Red/Black   
1  Set of 10 Battery Operated Multi LED Train Chr...   
2         ViewSonic Pro8200 DLP Multimedia Projector   
3  Concept Housewares WR-44526 Solid-Wood Ceiling...   

                                 product_description  median_relevance  \
0  Red satin accent pillow embroidered with a hea...                 1   
1  Set of 10 Battery Operated Train Christmas Lig...                 4   
2                                                NaN                 4   
3  Like a silent and sturdy tree, the Southern En...                 4   

   relevance_variance  
0               0.000  
1               0.000  
2               0.471  
3               0.000  


In [31]:
print(test_set.head(20))

    id                           query  \
0    3                electric griddle   
1    6           phillips coffee maker   
2    9             san francisco 49ers   
3   11                  aveeno shampoo   
4   12  flea and tick control for dogs   
5   14                     table clock   
6   15                  infinity scarf   
7   16       workout clothes for women   
8   18                  lego star wars   
9   19                kitchenaid mixer   
10  21                  notebook paper   
11  22              headphones earbuds   
12  23               16 gb memory card   
13  24         playstation vita system   
14  25                  cleaning robot   
15  26                      toy trucks   
16  27              converse high tops   
17  29            baseball photo frame   
18  30                       wine rack   
19  33              pittsburgh pirates   

                                        product_title  \
0                     Star-Max 48 in Electric Griddle   
1  

In [32]:
print(sample_submission_set.head(20))

    id  prediction
0    3           3
1    6           3
2    9           3
3   11           3
4   12           3
5   14           3
6   15           3
7   16           3
8   18           3
9   19           3
10  21           3
11  22           3
12  23           3
13  24           3
14  25           3
15  26           3
16  27           3
17  29           3
18  30           3
19  33           3


In [33]:
print(len(train_set))
print(len(test_set))
print(len(sample_submission_set))

10158
22513
22513


In [34]:
nlp = spacy.load("en_core_web_sm")

In [35]:
print(nlp.Defaults.stop_words)

{'ca', 'anyway', 'would', 'among', 'well', 'where', 'themselves', 'twelve', 'more', 'what', 'serious', 'front', '8', 'thereby', 'become', 'us', 'everywhere', 'noone', 'as', 'four', '‘s', 'last', 'therein', 'thereupon', 'am', 'will', 'they', 'already', 'nine', 'from', 'very', 'them', 'may', 'often', 'however', 'no', 'on', 'you', 'empty', 'if', 'side', 'besides', 'eight', 'except', 'of', 'seems', 'around', 'we', 'take', 'down', 'anywhere', 'indeed', 'i', 'throughout', 'nothing', 'thereafter', 'others', 'becomes', 'since', 'seeming', 'sometime', 'bottom', 'in', 'n’t', 'go', 'nor', 'that', 'almost', 'sixty', 'only', 'up', 'herein', 'thence', 'three', 'towards', 'meanwhile', 'used', 'your', 'an', 'give', 'out', 'thus', 'do', 'any', 'whereupon', 'full', 'those', 'our', 'the', 'whoever', '’ve', 'wherein', 'a', '‘d', 'made', 'forty', 'she', '‘re', 'were', 'few', 'becoming', 'due', 'somewhere', 'perhaps', 'yet', 'elsewhere', 'he', 'might', 'please', 'who', 'third', 'each', 'top', 'over', 'ten',

In [36]:
for i in range(0, 10):
    nlp.Defaults.stop_words.add(str(i))

In [37]:
print(nlp.Defaults.stop_words)

{'ca', 'anyway', 'would', 'among', 'well', 'where', 'themselves', 'twelve', 'more', 'what', 'serious', 'front', '8', 'thereby', 'become', 'us', 'everywhere', 'noone', 'as', 'four', '‘s', 'last', 'therein', 'thereupon', 'am', 'will', 'they', 'already', 'nine', 'from', 'very', 'them', 'may', 'often', 'however', 'no', 'on', 'you', 'empty', 'if', 'side', 'besides', 'eight', 'except', 'of', 'seems', 'around', 'we', 'take', 'down', 'anywhere', 'indeed', 'i', 'throughout', 'nothing', 'thereafter', 'others', 'becomes', 'since', 'seeming', 'sometime', 'bottom', 'in', 'n’t', 'go', 'nor', 'that', 'almost', 'sixty', 'only', 'up', 'herein', 'thence', 'three', 'towards', 'meanwhile', 'used', 'your', 'an', 'give', 'out', 'thus', 'do', 'any', 'whereupon', 'full', 'those', 'our', 'the', 'whoever', '’ve', 'wherein', 'a', '‘d', 'made', 'forty', 'she', '‘re', 'were', 'few', 'becoming', 'due', 'somewhere', 'perhaps', 'yet', 'elsewhere', 'he', 'might', 'please', 'who', 'third', 'each', 'top', 'over', 'ten',

In [38]:
#zamjena svih praznina s jednim praznim znakom
#gledaj samo tokene koji nisu stopwords i koji nisu punktacijski znakovi
#napisi sve tokene malim slovima 

#mozda omogucit da se svi numericki znakovi zamjenjuju slovima?

def preprocess_text(texts):
    all_preprocessed = []
    
    for tup in texts.items():
        preprocessed_text = []
        text = tup[1]

        if isinstance(text, float):
            preprocessed_text.append([""])
        
        else:
            text = ' '.join(text.split())
            
            doc = nlp(text)

            for sent in doc.sents: 
                for token in sent:
                    if not token.is_stop and not token.is_punct and not any(map(str.isdigit, str(token))):
                        preprocessed_text.append(str.lower(token.lemma_))

        all_preprocessed.append(" ".join(map(str, preprocessed_text)))
        
    return all_preprocessed
        


In [39]:
#spajamo naslov i opis produkta u 1 stupac kako bi ih mogli zajedno pretprocesirat

df_preprocessed = pd.DataFrame(train_set)
df_preprocessed = df_preprocessed.replace(np.nan, '', regex=True)
df_preprocessed['product'] = df_preprocessed["product_title"] + " " + df_preprocessed["product_description"]

In [40]:
#preprocesiraj posebno query i posebno product

df_preprocessed["product"] = preprocess_text(df_preprocessed["product"])
df_preprocessed["query"] = preprocess_text(df_preprocessed["query"])

In [41]:
print(df_preprocessed["product"][1])

set battery operated multi led train christmas lights clear wire set battery operate train christmas lights item feature color multi color bulb match train light cover clear wire multi color consist red green blue yellow bulb number bulb string bulb size micro led space bulb inch lighted length foot total length foot inch lead cord additional product feature led light use energy cool touch bulb burn rest stay lit light equip lamp lock feature make replaceable interchangeable keep fall require aa battery include convenient timer switch locate battery pack timer function battery pack allow hour hour connect multiple set ul list indoor use train dimension x x material(s plastic wire acrylic


In [43]:
print(df_preprocessed["query"][1])

lead christmas light


In [44]:
del df_preprocessed["product_title"]
del df_preprocessed["product_description"]

In [45]:
df_preprocessed.to_csv(r'./Dataset/dataframes/df_preprocessed_without_numbers.csv', encoding='utf-8', header='true')